In [26]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [34]:
from tensorflow.keras import datasets 
from tensorflow.keras import layers 
from tensorflow.keras import activations 
from tensorflow.keras import initializers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers 
from tensorflow.keras import regularizers 
from tensorflow.keras.optimizers import schedules 
from tensorflow.keras import callbacks 
from tensorflow.keras import utils
from tensorflow.keras import models 

In [2]:
#reading input from local

df = pd.read_csv('Combined.csv')

C:\Users\desktop\AppData\Local\Temp\ipykernel_11432\2010559339.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Combined.csv')


In [3]:
df.dtypes

Unnamed: 0       int64
Seq              int64
Dur            float64
RunTime        float64
Mean           float64
Sum            float64
Min            float64
Max            float64
Proto           object
sTos           float64
dTos           float64
sDSb            object
dDSb            object
sTtl           float64
dTtl           float64
sHops          float64
dHops          float64
Cause           object
TotPkts          int64
SrcPkts          int64
DstPkts          int64
TotBytes         int64
SrcBytes         int64
DstBytes         int64
Offset           int64
sMeanPktSz     float64
dMeanPktSz     float64
Load           float64
SrcLoad        float64
DstLoad        float64
Loss             int64
SrcLoss          int64
DstLoss          int64
pLoss          float64
SrcGap         float64
DstGap         float64
Rate           float64
SrcRate        float64
DstRate        float64
State           object
SrcWin         float64
DstWin         float64
sVid           float64
dVid       

In [4]:
null_col = [key for key in dict(df.isnull().sum().sort_values(ascending=False))
              if dict(df.isnull().sum().sort_values(ascending=False))[key] > 937218]
print(null_col)


['dVid', 'sVid', 'DstWin', 'DstTCPBase', 'SrcWin', 'dDSb', 'dHops', 'dTtl', 'dTos', 'SrcTCPBase', 'DstGap', 'SrcGap']


In [5]:
df.isnull().sum().sort_values(ascending=False)

dVid           1213881
sVid           1101319
DstWin         1038812
DstTCPBase      985843
SrcWin          973470
dDSb            943067
dHops           943067
dTtl            943067
dTos            943067
SrcTCPBase      937219
DstGap          937219
SrcGap          937219
sTos               214
sDSb               214
sTtl               214
sHops              214
DstRate              0
Rate                 0
SrcRate              0
pLoss                0
DstLoss              0
Unnamed: 0           0
State                0
Loss                 0
TcpRtt               0
SynAck               0
AckDat               0
Label                0
Attack Type          0
SrcLoss              0
dMeanPktSz           0
DstLoad              0
TotPkts              0
Dur                  0
RunTime              0
Mean                 0
Sum                  0
Min                  0
Max                  0
Proto                0
Cause                0
SrcPkts              0
SrcLoad              0
DstPkts    

In [6]:
#dropping column with greater null values
df.drop(columns = null_col,inplace = True)

In [7]:
#remaing null columns with less number of null values to be filled with zero
null_col_2 = [key for key in dict(df.isnull().sum().sort_values(ascending=False))
              if dict(df.isnull().sum().sort_values(ascending=False))[key] > 0]
print(null_col_2)

['sTtl', 'sHops', 'sTos', 'sDSb']


In [8]:
df.loc[:, null_col_2] = df.loc[:, null_col_2].fillna(0)
df.drop(columns = 'Unnamed: 0',inplace = True)

In [9]:
#pearson correlation
pear_cor = df.corr(method = 'pearson')
pear_cor = pd.DataFrame(pear_cor)
pear_cor

C:\Users\desktop\AppData\Local\Temp\ipykernel_11432\3979310324.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  pear_cor = df.corr(method = 'pearson')


,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,sTtl,sHops,...,Loss,SrcLoss,DstLoss,pLoss,Rate,SrcRate,DstRate,TcpRtt,SynAck,AckDat
Seq,1.000000,-0.028287,-0.028287,-0.028287,-0.028287,-0.028287,-0.028287,-0.060186,-0.205341,-0.266175,...,-0.057305,-0.049006,-0.035669,-0.060023,-0.007866,-0.007485,-0.005923,-0.187764,-0.036616,-0.265220
Dur,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
RunTime,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
Mean,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
Sum,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
Min,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
Max,-0.028287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.028247,-0.204560,-0.185770,...,0.083293,0.037262,0.090290,0.021467,-0.008797,-0.008514,-0.007119,0.075002,0.026364,0.091397
sTos,-0.060186,-0.028247,-0.028247,-0.028247,-0.028247,-0.028247,-0.028247,1.000000,0.176174,0.003810,...,0.030642,0.046777,-0.004271,0.043443,0.010380,-0.000732,-0.000609,-0.018499,-0.003076,-0.026789
sTtl,-0.205341,-0.204560,-0.204560,-0.204560,-0.204560,-0.204560,-0.204560,0.176174,1.000000,0.391363,...,-0.015384,-0.003737,-0.020228,-0.009363,-0.000405,-0.003036,-0.002779,-0.088156,-0.014575,-0.127764
sHops,-0.266175,-0.185770,-0.185770,-0.185770,-0.185770,-0.185770,-0.185770,0.003810,0.391363,1.000000,...,-0.021188,-0.010308,-0.021914,-0.013571,-0.000418,-0.000077,-0.001804,-0.097117,-0.017289,-0.139225


In [10]:
# List of columns that are highky corelated
threshold = 0.9

highly_correlated = []

for i in range(len(pear_cor.columns)):
  for j in range(i+1,len(pear_cor.columns)):
    if abs(pear_cor.iloc[i,j]) >= threshold:
      highly_correlated.append((pear_cor.columns[i],pear_cor.columns[j]))


highly_correlated = set([col for pair in highly_correlated for col in pair])

highly_related_column_list = list(highly_correlated)

In [11]:
#dataframe with highly correlated columns
df_high_corr = df[highly_related_column_list]
# df_high_corr['Label'] = df['Label']
df_high_corr

,SrcPkts,DstBytes,Rate,Min,Load,RunTime,SrcRate,Sum,DstRate,SrcLoad,SrcBytes,Mean,TotPkts,DstPkts,Max,TotBytes,Dur,DstLoad
0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,98,0.000000,1,0,0.000000,98,0.000000,0.000000
1,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,98,0.000000,1,0,0.000000,98,0.000000,0.000000
2,196,4881,42.616875,4.998020,396280.125000,4.998020,39.015450,4.998020,3.401347,388901.187500,244212,4.998020,214,18,4.998020,249093,4.998020,7378.921875
3,163,5467,36.614376,4.998037,352339.937500,4.998037,32.412727,4.998037,4.001571,344005.468800,216245,4.998037,184,21,4.998037,221712,4.998037,8334.472656
4,204,4493,44.404858,4.999453,445855.156300,4.999453,40.604443,4.999453,3.600394,439043.218800,275723,4.999453,223,19,4.999453,280216,4.999453,6811.945312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215885,1,98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102,0.000000,2,1,0.000000,200,0.000000,0.000000
1215886,3,2766,21.221781,0.235607,69199.984380,0.235607,8.488712,0.235607,8.488712,6587.240723,290,0.235607,6,3,0.235607,3056,0.235607,62612.742190
1215887,2,92,20.014610,0.099927,6404.675293,0.099927,10.007305,0.099927,0.000000,6404.675293,160,0.099927,3,1,0.099927,252,0.099927,0.000000
1215888,3,290,3.823062,1.307852,2434.526123,1.307852,1.529225,1.307852,1.529225,1247.847534,306,1.307852,6,3,1.307852,596,1.307852,1186.678589


In [12]:
#normalization min max scalling
scaler = MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df_high_corr),columns = df_high_corr.columns)

scaled['Label'] = df['Label']
scaled

,SrcPkts,DstBytes,Rate,Min,Load,RunTime,SrcRate,Sum,DstRate,SrcLoad,SrcBytes,Mean,TotPkts,DstPkts,Max,TotBytes,Dur,DstLoad,Label
0,0.001014,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000073,0.000000,0.000000,0.000000,0.000000,0.000012,0.000000,0.000000e+00,Benign
1,0.001014,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000073,0.000000,0.000000,0.000000,0.000000,0.000012,0.000000,0.000000e+00,Benign
2,0.198783,0.001072,1.522031e-06,0.250897,1.506769e-06,0.250897,7.803090e-06,0.250897,1.308210e-07,6.404829e-05,0.181776,0.250897,0.053303,0.005712,0.250897,0.053488,0.250897,2.805674e-08,Benign
3,0.165314,0.001201,1.307656e-06,0.250898,1.339696e-06,0.250898,6.482545e-06,0.250898,1.539066e-07,5.665439e-05,0.160959,0.250898,0.045796,0.006665,0.250898,0.047607,0.250898,3.169001e-08,Benign
4,0.206897,0.000987,1.585888e-06,0.250969,1.695267e-06,0.250969,8.120889e-06,0.250969,1.384767e-07,7.230620e-05,0.205231,0.250969,0.055556,0.006030,0.250969,0.060172,0.250969,2.590093e-08,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215885,0.001014,0.000022,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000076,0.000000,0.000250,0.000317,0.000000,0.000034,0.000000,0.000000e+00,Benign
1215886,0.003043,0.000608,7.579207e-07,0.011827,2.631178e-07,0.011827,1.697742e-06,0.011827,3.264889e-07,1.084855e-06,0.000216,0.011827,0.001251,0.000952,0.011827,0.000647,0.011827,2.380713e-07,Benign
1215887,0.002028,0.000020,7.148075e-07,0.005016,2.435238e-08,0.005016,2.001461e-06,0.005016,0.000000e+00,1.054788e-06,0.000119,0.005016,0.000501,0.000317,0.005016,0.000045,0.005016,0.000000e+00,Benign
1215888,0.003043,0.000064,1.365379e-07,0.065653,9.256753e-09,0.065653,3.058450e-07,0.065653,5.881635e-08,2.055085e-07,0.000228,0.065653,0.001251,0.000952,0.065653,0.000119,0.065653,4.512086e-09,Benign


In [14]:
scaled['Label'].unique()

array(['Benign', 'Malicious'], dtype=object)

In [61]:
#test train split 
X = scaled.copy()
y = scaled[['Label']]
X.drop(columns = 'Label',axis = 1,inplace = True)


In [62]:
from sklearn.preprocessing import LabelEncoder

# Suppose 'labels' is a list of string labels
labels = ['Benign', 'Malicious']

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder on the labels and transform labels to integers
y = label_encoder.fit_transform(y)

# Now 'integer_labels' contains integer representations of labels
print(y)

[0 0 0 ... 0 0 0]


A:\Anacondainstall\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [63]:
 X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.32 , random_state=42)

In [64]:
 X_train.shape

(826805, 18)

In [69]:
#implementing ann
model = tf.keras.Sequential([
    tf.keras.Input(18,),
    layers.Dense(units = 128, activation = activations.relu,kernel_initializer='glorot_uniform', kernel_regularizer=None),
    layers.Dense(units = 64, activation = activations.relu,kernel_initializer='glorot_uniform', kernel_regularizer=None),
    layers.Dense(units = 32, activation = activations.relu,kernel_initializer='glorot_uniform', kernel_regularizer=None),
    layers.Dense(1, activation='sigmoid')
])

In [70]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               2432      
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 1)                 33        
                                                                 
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________


In [71]:

model.compile(
    
    # optimizer to use
    optimizer=optimizers.SGD(learning_rate=0.01), # method 1
    # optimizer='SGD', # method 2
    
    # loss metric to optimize 
    # Use BinaryCrossentropy when there are only two label classes (assumed to be 0 and 1);
    # Use CategoricalCrossentropy when there are two or more label classes (OHE);
    # Use SparseCategoricalCrossentropy where there are two or more label classes (integer)
    loss=losses.SparseCategoricalCrossentropy(), # method 1
    # loss='sparse_categorical_crossentropy', # method 2
    
    
    # additional metrics to track (but not optimize) 
    metrics=['accuracy']
)  

In [ ]:
%%time

# TRAIN model 

# if you'd like to implement Early Stopping, you can create a callback for it
# es_callback = callbacks.EarlyStopping(monitor='val_loss', patience=100)

history = model.fit(
    
    # training set images
    X_train, 
    
    # training set labels
    y_train, 
    
    # number of epochs to train
    epochs=10,
    
    # mini-batch SGD batch size
    # if None, defaults to 32
    batch_size=32,
    
    # specify your callbacks
    # callbacks=[es_callback],
    
    # validation set - can use validation_split or validation_data
    validation_split=0.1
) 

Epoch 1/10
10195/23254 [============>.................] - ETA: 25s - loss: nan - accuracy: 0.3927

In [52]:
tf_dataset()

NameError: name 'tf_dataset' is not defined